In [ ]:
#
# Ruler Estimation 
# Author: Abhir Bhalerao, Department of Computer Science, University of Warwick, Coventry, UK
# 28th September, 2021
# abhir.bhalerao@warwick.ac.uk
#
# This notebook presents an approximate ruler graduations/spacing estimator based on the published work:
#
# Ruler Detection for Autoscaling Forensic Images, Abhir Bhalerao and Greg Reynolds. 
# International Journal of Digital Crime and Forensics, 
# Volume 6, Issue 1, 2014. Pages 9-27.
# https://www.researchgate.net/publication/264277138_Ruler_Detection_for_Autoscaling_Forensic_Images
#
# For the wavelength/spacing estimation, it uses the YIN pitch detector -- please see de Cheveign´e, A. and Kawahara, H. (2002). 
# YIN, a fundamental frequency estimator for speech and music. Journal of the Acoustical Society of America, 111(4):1917–1930.
#
# The code is meant to be illustrative and not definitive. 
#
# THIS CODE IS PROVIDED “AS IS” AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING THE IMPLIED WARRANTIES 
# OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. 
# IN NO EVENT SHALL THE AUTHOR OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, 
# INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES 
# (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, 
# OR PROFITS; OR BUSINESS INTERRUPTION) SUSTAINED BY YOU OR A THIRD PARTY, 
# HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, 
# STRICT LIABILITY, OR TORT ARISING IN ANY WAY OUT OF THE USE OF THIS SAMPLE CODE, 
# EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE
#
# If you do use this code, please leave these notices and/or acknowledge the original code/cite our paper, 
# many thanks and we hope you find it useful!

In [ ]:
#
# How to use the code.
#
# 1. All the important stuff happens in the Ruler class. It takes an square image block (this should be
#    a single channel, e.g. grayscale). This block should be chosen to contain part of a ruler in the image. 
# 2. The angle of the ruler, the spacing of the graduations and the relative offset or phase of the graduations
#    are estimated. For must purposes, only the spacing is important as this allows for the image to be
#    calibrated. The spacing is returned in pixels.
# 3. By default, the smallest ruler spacing is given, but the 'which' parameter to the Ruler.wavelength() method
#    can be changed to 1, 2, ... to return multiple spacings. Usually these are just multiple of the smallest.
# 4. The Ruler.orientation() method should be called first, then the Ruler.wavelength() and then Ruler.phase()
#
# 5. The helper functions hp_filter() and extract_block() can be used to pre-process the input (HP filter it), and 
#    extract a suitable block from the input.
# 
# For more information on why the code does what it does, see our paper. Note that *not* all the refimements 
# presented in the published work are implemented in the below. Interesting intermediate results are stored in
# instance members and can be easily plotted, e.g. self.corr, self.spatial, self.csdf, self.sine_rotated, etc.
#
# 6. Interactive version. A simple interactive (OpenCV/mathplotlib.pyplot) driver for the class is provided with
#    this distribution: 
#    
#    Usage: 
#        $ python ruler-interactive.py <image-filename>
# 
#    Once running, you can click on a point in the display window and that should extract an image block and
#    show the results in the plots window. You can use the fast-keys, [], to increase/decrease the block size 
#    (these are trapped by the figure_on_press method).
#    To exit, you need to close the figure window first, then any key will exit the program.  OpenCV windows and matplotlib windows don't play well
#    together, unfortunately.
#
# Possible improvements to the ruler estimation:
#
# The ruler offset (phase) is sensitive to the wavelength estimate, so a local search around the approximation 
# would be beneficial, e.g. a simplex search using a MSE cost (input v. synthesized sine grating model). The ruler 
# orientation may be found incorrectly as it use the principal axis of the amp spectrum. If the ruler edge 'energy' 
# dominates the chosen block window, the angle will be 90 degrees out! 
# It is possible to differentiate the two but this requires looking at the line spectra more closely.
#
# 
# AB 28/09/2021 

In [ ]:
from preproc import *
from ruler import *

In [ ]:
# Example use

# 1. Load an image which has a ruler in it
im = cv2.imread('./EXAMPLES/ruler3-300.png')
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

fig = plt.figure(figsize=(7,10))
plt.imshow(im)
plt.show()

# 2. Convert to gray-scale, if not already
# gray scale
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

# 3. HP filter as a pre-process
#
hp = hp_filter(gray)

# 4. Locate a smallish region on the ruler where you want to take an estimate of the spacing of graduations
#
X = 766
Y = 463

# 5. Select a block size
block_size = 128

# 6. Extract the block from the HP filtered input
block = extract_block(hp, X, Y, block_size)

plt.imshow(block)
plt.show()


In [ ]:
# 7. Instantiate a Ruler object, passing the extract image block
ruler = Ruler(block)

plt.imshow(ruler.block), plt.title('ruler block')
plt.show()


# 8. Calculate estiamte of the ruler angle first
angle = ruler.orientation()
plt.imshow(ruler.corr), plt.title('ruler corr')
plt.show()
print('angle is ', np.rad2deg(angle))

# 9. Now we can estimate the graduation spacing and offset
spacing = ruler.wavelength(which=0)
offset = ruler.phase(invert= -1)  # note the 'invert' argument for whether the ruler is white or black

print('spacing is ', spacing, ', offset is ', offset)

# all done, but you may want to see what it's done...

In [ ]:


# 10. this is how overlay the results on to the input using matplotlib.pyplot
# you may want to do similar to place graudations on the original input

fig, ax = plt.subplots(1, figsize=(9,9))
plt.imshow(block, 'gray')

ruler.show_grads(ax, block_size, ruler.angle, ruler.spacing, ruler.offset)

plt.show()